# anaconda::accelerate::cusparse

https://docs.continuum.io/accelerate/cusparse

In [1]:
import numpy as np
import accelerate.cuda.sparse as cusp

In [5]:
R = np.load('R.npy')
R_sp = cusp.csr_matrix(R)
Dinv = np.load('Dinv.npy')[:,0]

R_sp.shape

(40000, 40000)

In [3]:
N = R.shape[0] # 行列サイズ

handl = cusp.Sparse() # Sparseクラスのオブジェクト
descr = handl.matdescr() # 行列ディスクリプタ。対称行列とか指定可能っぽい

y = alpha \* trans(A).dot(x) + beta \* y

http://qiita.com/aokomoriuta/items/079a63c91bb3531df43d

http://docs.nvidia.com/cuda/cusparse/index.html#cusparsecreatematdescr

In [9]:
%%timeit
handl.csrmv( 
    trans='N', #転置とか共役とか。N:そのまま
    m=N, n=N, 
    nnz = R_sp.nnz, 
    alpha=1.0, 
    descr=descr, 
    csrVal=R_sp.data, csrRowPtr=R_sp.indptr, csrColInd=R_sp.indices,  #疎行列の内容
    x=Dinv, 
    beta=0.0, y=Dinv)

The slowest run took 4.25 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 595 µs per loop


時間計測の結果、scipy.sparseの半分くらいの時間で出来ているっぽい。

Dinvがnumpy.ndarrayなのは重要。

うまく行けばGPU利用により２倍程度の高速化は可能か。

In [10]:
descr = handl.matdescr(
    indexbase = 0, # 0 for 0-based indexing, or 1 for 1-based indexing
    diagtype = 'N', #  ‘N’ : diagonal has non-unit elements. ‘U’ : diagonal only contains unit elements.
    fillmode = 'L',
    
) 